In [1]:
import pickle 
import pandas as pd 
import numpy as np

In [2]:
#Load in scraped apartments 
with open('../data/data_v2.pickle','rb') as f:
    buildings = pickle.load(f)

# Clean out scraped data for apartment info

In [3]:
apartments = pd.DataFrame()
for b in buildings: 
    if b and 'apts' in b.keys():
        apt_list = b['apts']
        amenities = b['amenities']
        i = 1
        for a in apt_list: 
            price_df = a['price_history'][0]
            price_df['address'] = b['address']
            price_df['BIN'] = b['BIN']
            room, bed, bath = None, None, None
            for deats in a['details']:
                if 'room' in deats:
                    room = deats
                elif ('studio' in deats) or ('bed' in deats): 
                    bed = deats
                elif 'bath' in deats: 
                    bath = deats 
                    
            price_df['rooms'] = room
            price_df['bedrooms'] = bed
            price_df['bath'] = bath
            price_df['apt_num'] = i
            price_df['amenities'] = str(amenities)
            apartments = apartments.append(price_df)
            i += 1
            
        

In [7]:
apartments.columns

Index([0, 1, 2, 'address', 'BIN', 'rooms', 'bedrooms', 'bath', 'apt_num',
       'amenities'],
      dtype='object')

In [4]:
apartments.rename(columns = {0:'date', 1:'descr', 2:'rent_price'}, inplace = True)

In [6]:
apartments.head()

,date,descr,rent_price,address,BIN,rooms,bedrooms,bath,apt_num,amenities
0,10/02/2020,Off market temporarily,"$2,750",279+lee+avenue+brooklyn,3061407.0,4 rooms,3 beds,1 bath,1,"['Live-in Super', 'Virtual Doorman', 'NYC Stor..."
1,10/01/2020,Listing entered contract,"$2,750",279+lee+avenue+brooklyn,3061407.0,4 rooms,3 beds,1 bath,1,"['Live-in Super', 'Virtual Doorman', 'NYC Stor..."
2,09/25/2020,Listed by StayBK LLC,"$2,750",279+lee+avenue+brooklyn,3061407.0,4 rooms,3 beds,1 bath,1,"['Live-in Super', 'Virtual Doorman', 'NYC Stor..."
0,04/23/2017,Listing rented,"$2,199",279+lee+avenue+brooklyn,3061407.0,4 rooms,2 beds,1 bath,2,"['Live-in Super', 'Virtual Doorman', 'NYC Stor..."
1,03/07/2017,Listing entered contract,"$2,199",279+lee+avenue+brooklyn,3061407.0,4 rooms,2 beds,1 bath,2,"['Live-in Super', 'Virtual Doorman', 'NYC Stor..."


# Get apartments that have a date before 2018 and after

In [5]:
#Convert to datetime and create a year variable 
apartments['date'] = pd.to_datetime(apartments['date'])

In [6]:
apartments['year'] = apartments['date'].apply(lambda x: x.year)

In [7]:
#take apartments with dates in 2017/2016 and 2019 only 
apartments_year_subset = apartments[((apartments['year'] >= 2016.0)&(apartments['year'] <= 2017.0))| (apartments['year'] == 2019.0)]

In [8]:
apartments_grouped = apartments_year_subset.groupby(['BIN','apt_num']).filter(lambda x: x['year'].min() < 2018 and x['year'].max() >= 2019)

In [9]:
len(apartments_grouped['BIN'].unique())

123

In [10]:
len((apartments_grouped['BIN']+apartments_grouped['apt_num']).unique())

262

# Combine with proximity and borough information

In [11]:
housing_df = pd.read_csv('../data/test_set.csv')

In [11]:
housing_df.columns

Index(['RegistrationID', 'BuildingID', 'BoroID', 'Boro', 'HouseNumber',
       'LowHouseNumber', 'HighHouseNumber', 'StreetName', 'StreetCode', 'Zip',
       'Block', 'Lot', 'BIN', 'CommunityBoard', 'LastRegistrationDate',
       'RegistrationEndDate', 'ADDRESS_ID', 'H_NO', 'the_geom', 'HNO_SUFFIX',
       'HYPHEN_TYP', 'SIDE_OF_ST', 'SPECIAL_CO', 'BOROCODE', 'ZIPCODE',
       'CREATED', 'MODIFIED', 'ST_NAME', 'HN_RNG_SUF', 'HN_RNG', 'PHYSICALID',
       'PRE_MODIFI', 'PRE_DIRECT', 'PRE_TYPE', 'POST_TYPE', 'POST_DIREC',
       'POST_MODIF', 'FULL_STREE', 'geometry', '250m_to_2018_dev',
       'no_proximity_newdev', 'lat', 'lon'],
      dtype='object')

In [12]:
housing_df

,RegistrationID,BuildingID,BoroID,Boro,HouseNumber,LowHouseNumber,HighHouseNumber,StreetName,StreetCode,Zip,...,PRE_TYPE,POST_TYPE,POST_DIREC,POST_MODIF,FULL_STREE,geometry,250m_to_2018_dev,no_proximity_newdev,lat,lon
0,225688,898489,2,BRONX,1743,1743,1745,DAVIDSON AVENUE,23520,10453.0,...,NaN,AVE,NaN,NaN,DAVIDSON AVE,POINT (-73.91319211844298 40.84837018078486),True,False,-73.913192,40.848370
1,360476,354310,3,BROOKLYN,2578,2578,2578,PITKIN AVENUE,70930,11208.0,...,NaN,ST,NaN,NaN,MILFORD ST,POINT (-73.87693709893625 40.67447780179333),True,False,-73.876937,40.674478
2,341209,216247,3,BROOKLYN,909,909,909,BUSHWICK AVENUE,23930,11221.0,...,NaN,AVE,NaN,NaN,BUSHWICK AVE,POINT (-73.92455362966729 40.69335103843348),True,False,-73.924554,40.693351
3,119035,26177,1,MANHATTAN,76,76,76,ORCHARD STREET,27590,10002.0,...,NaN,ST,NaN,NaN,ORCHARD ST,POINT (-73.99033507924125 40.71770738191128),True,False,-73.990335,40.717707
4,369220,339320,3,BROOKLYN,99,99,99,MONROE STREET,61730,11216.0,...,NaN,ST,NaN,NaN,MONROE ST,POINT (-73.95496530056866 40.68508748059449),True,False,-73.954965,40.685087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,351132,263486,3,BROOKLYN,143,143,143,EAST 42 STREET,36730,11203.0,...,NaN,ST,NaN,NaN,E 42 ST,POINT (-73.9380395532623 40.65234068363606),False,True,-73.938040,40.652341
1996,217785,46850,2,BRONX,2932,2932,2932,BAISLEY AVENUE,10120,10461.0,...,NaN,AVE,NaN,NaN,BAISLEY AVE,POINT (-73.82858106234812 40.83580481032219),False,True,-73.828581,40.835805
1997,142028,3331,1,MANHATTAN,832,832,832,6 AVENUE,10510,10001.0,...,AVE,NaN,NaN,NaN,AVE OF THE AMERICAS,POINT (-73.98996627725042 40.74649046908136),False,True,-73.989966,40.746490
1998,408417,810655,4,QUEENS,111-45,111-45,111-45,76 AVENUE,15840,11375.0,...,NaN,BLVD,NaN,NaN,QUEENS BLVD,POINT (-73.83639625540019 40.71767440016517),False,True,-73.836396,40.717674


In [12]:
apart_join = apartments_grouped.merge(housing_df[['BIN', 'no_proximity_newdev', 'BoroID','Boro']], on = ['BIN'])

In [13]:
apart_join

,date,descr,rent_price,address,BIN,rooms,bedrooms,bath,apt_num,amenities,year,no_proximity_newdev,BoroID,Boro
0,2019-06-24,"Delisted by StayBK LLC. Last priced at $1,899","$1,899",279+lee+avenue+brooklyn,3061407.0,2 rooms,studio,1 bath,4,"['Live-in Super', 'Virtual Doorman', 'NYC Stor...",2019.0,True,3,BROOKLYN
1,2019-06-07,Later Listed by StayBK LLC,"$1,999",279+lee+avenue+brooklyn,3061407.0,2 rooms,studio,1 bath,4,"['Live-in Super', 'Virtual Doorman', 'NYC Stor...",2019.0,True,3,BROOKLYN
2,2016-10-02,Next Door Realty NYC Listing rented,"$1,900",279+lee+avenue+brooklyn,3061407.0,2 rooms,studio,1 bath,4,"['Live-in Super', 'Virtual Doorman', 'NYC Stor...",2016.0,True,3,BROOKLYN
3,2016-08-05,Previously Listed by Next Door Realty NYC,"$1,900",279+lee+avenue+brooklyn,3061407.0,2 rooms,studio,1 bath,4,"['Live-in Super', 'Virtual Doorman', 'NYC Stor...",2016.0,True,3,BROOKLYN
4,2019-10-28,Price decreased by 1%,"$2,179 ↓",4469+broadway+manhattan,1064467.0,3 rooms,1 bed,1 bath,4,"['Gym', 'Laundry in Building', 'Live-in Super'...",2019.0,True,1,MANHATTAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1569,2017-12-04,Previously Listed by ArchRock LLC,"$2,795",1575+lexington+avenue+manhattan,1051813.0,2 rooms,2 beds,1 bath,6,['NYC Storm Zone 5'],2017.0,False,1,MANHATTAN
1570,2017-11-28,ArchRock LLC Listing is no longer available on...,"$2,799",1575+lexington+avenue+manhattan,1051813.0,2 rooms,2 beds,1 bath,6,['NYC Storm Zone 5'],2017.0,False,1,MANHATTAN
1571,2017-11-07,Previously Listed by ArchRock LLC,"$2,799",1575+lexington+avenue+manhattan,1051813.0,2 rooms,2 beds,1 bath,6,['NYC Storm Zone 5'],2017.0,False,1,MANHATTAN
1572,2016-10-16,Next Door Realty NYC Listing is no longer avai...,"$2,500",1575+lexington+avenue+manhattan,1051813.0,2 rooms,2 beds,1 bath,6,['NYC Storm Zone 5'],2016.0,False,1,MANHATTAN


In [14]:
apart_join["rent_price_cleaned"] = apart_join["rent_price"].apply(lambda x: int(x[1:6].replace(",","")))

In [12]:
#All apartments that match the criteria
apart_join.to_csv("../data/all_apartments.csv")


# Get one price before 2018 and one price after for each apartment

In [15]:
apart_join['max_date'] = apart_join[apart_join['year'] < 2018].groupby(['BIN','apt_num'])['date'].transform(max)

In [16]:
apart_join['ave_2019'] = apart_join[apart_join['year'] == 2019].groupby(['BIN','apt_num'])["rent_price_cleaned"].transform(np.mean)

In [17]:
prices_before = apart_join[~ np.isnat(apart_join['max_date'])]

In [41]:
prices_before['keep_row'] = prices_before['max_date'].apply(lambda x: 1 if x == prices_before[])

,date,descr,rent_price,address,BIN,rooms,bedrooms,bath,apt_num,amenities,year,no_proximity_newdev,BoroID,Boro,rent_price_cleaned,max_date,ave_2019
2,2016-10-02,Next Door Realty NYC Listing rented,"$1,900",279+lee+avenue+brooklyn,3061407.0,2 rooms,studio,1 bath,4,"['Live-in Super', 'Virtual Doorman', 'NYC Stor...",2016.0,True,3,BROOKLYN,1900,2016-10-02,NaN
3,2016-08-05,Previously Listed by Next Door Realty NYC,"$1,900",279+lee+avenue+brooklyn,3061407.0,2 rooms,studio,1 bath,4,"['Live-in Super', 'Virtual Doorman', 'NYC Stor...",2016.0,True,3,BROOKLYN,1900,2016-10-02,NaN
9,2016-07-11,Douglas Elliman Listing rented Last priced at ...,"$2,210",4469+broadway+manhattan,1064467.0,3 rooms,1 bed,1 bath,4,"['Gym', 'Laundry in Building', 'Live-in Super'...",2016.0,True,1,MANHATTAN,2210,2016-07-11,NaN
10,2016-05-18,Previously Listed by Douglas Elliman,"$2,260",4469+broadway+manhattan,1064467.0,3 rooms,1 bed,1 bath,4,"['Gym', 'Laundry in Building', 'Live-in Super'...",2016.0,True,1,MANHATTAN,2260,2016-07-11,NaN
13,2017-03-27,Wilk Real Estate Listing rented,"$4,000",3122+emmons+avenue+brooklyn,3394348.0,4 rooms,2 beds,2.5 baths,7,"['Gym', 'NYC Storm Zone 1', 'Doorman', 'Elevat...",2017.0,True,3,BROOKLYN,4000,2017-03-27,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1569,2017-12-04,Previously Listed by ArchRock LLC,"$2,795",1575+lexington+avenue+manhattan,1051813.0,2 rooms,2 beds,1 bath,6,['NYC Storm Zone 5'],2017.0,False,1,MANHATTAN,2795,2017-12-22,NaN
1570,2017-11-28,ArchRock LLC Listing is no longer available on...,"$2,799",1575+lexington+avenue+manhattan,1051813.0,2 rooms,2 beds,1 bath,6,['NYC Storm Zone 5'],2017.0,False,1,MANHATTAN,2799,2017-12-22,NaN
1571,2017-11-07,Previously Listed by ArchRock LLC,"$2,799",1575+lexington+avenue+manhattan,1051813.0,2 rooms,2 beds,1 bath,6,['NYC Storm Zone 5'],2017.0,False,1,MANHATTAN,2799,2017-12-22,NaN
1572,2016-10-16,Next Door Realty NYC Listing is no longer avai...,"$2,500",1575+lexington+avenue+manhattan,1051813.0,2 rooms,2 beds,1 bath,6,['NYC Storm Zone 5'],2016.0,False,1,MANHATTAN,2500,2017-12-22,NaN


In [18]:
prices_after = apart_join[np.isnat(apart_join['max_date'])]

In [19]:
final_df = pd.DataFrame(columns = prices_before.columns)

for i, row in prices_before.iterrows():
    if row['date'] == row['max_date']:
        
        final_df = final_df.append(row)

In [21]:
final_df = final_df.drop_duplicates(subset=['BIN', 'apt_num'], keep='last')

In [63]:
final_df = final_df.drop(columns = ['ave_2019', 'max_date'])

In [68]:
prices_after = prices_after.drop_duplicates(subset=['BIN', 'apt_num'], keep='last').drop(columns = ['rent_price_cleaned','max_date']).rename(columns={'ave_2019':'rent_price_cleaned'})

In [69]:
all_prices = final_df.append(prices_after)

In [73]:
pre_post_prices = all_prices.drop(columns = [ 'date','descr','rent_price','address'])

In [82]:
pre_post_prices['amenities_fl'] = pre_post_prices['amenities'].apply(lambda x: 1 if len(x) > 2 else 0)

In [104]:
pre_post_prices

,BIN,rooms,bedrooms,bath,apt_num,amenities,year,no_proximity_newdev,BoroID,Boro,rent_price_cleaned,amenities_fl
2,3061407.0,2,0,1,4,"['Live-in Super', 'Virtual Doorman', 'NYC Stor...",2016.0,True,3,BROOKLYN,1900,1
9,1064467.0,3,1,1,4,"['Gym', 'Laundry in Building', 'Live-in Super'...",2016.0,True,1,MANHATTAN,2210,1
13,3394348.0,4,2,2.5,7,"['Gym', 'NYC Storm Zone 1', 'Doorman', 'Elevat...",2017.0,True,3,BROOKLYN,4000,1
18,4002125.0,7,3,1.5,3,[],2017.0,True,4,QUEENS,2000,0
24,1018892.0,2,0,1,2,[],2017.0,True,1,MANHATTAN,1950,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1543,4431832.0,4,1,1,6,[],2019.0,True,4,QUEENS,1725,0
1547,3396713.0,4,2,2,7,"['Bike Room', 'Cold Storage', 'Green Building'...",2019.0,False,3,BROOKLYN,3295,1
1555,3108486.0,4,3,1,3,[],2019.0,True,3,BROOKLYN,2566.67,0
1560,3108486.0,5,3,1,7,[],2019.0,True,3,BROOKLYN,2575,0


# Percent difference analysis - not used 

In [2]:
perc_diff_col = apart_join.groupby(['BIN','apt_num']).apply(lambda df: (((df[df['year'] <=  2017.0]['rent_price_cleaned']).mean()) - ((df[df['year'] ==  2019.0]['rent_price_cleaned']).mean()))/((df[df['year'] <=  2017.0]['rent_price_cleaned']).mean()))

NameError: name 'apart_join' is not defined

In [41]:
perc_diff = pd.DataFrame(perc_diff_col).rename(columns = {0:'price_diff'})

In [42]:
perc_diff

price_diff
BIN       apt_num            
1003107.0 2         -0.009600
          7         -0.009600
1004303.0 2          0.096386
1004471.0 1         -0.091445
          2         -0.091445
...                       ...
4206527.0 1         -0.081967
4430914.0 1          0.022403
          4          0.066667
4431832.0 3          0.021277
          6          0.028169

[262 rows x 1 columns]

In [56]:
apart_join[['BIN', 'rooms', 'bedrooms','bath', 'apt_num', 'amenities', 'no_proximity_newdev', 'BoroID',
       'Boro']].drop_duplicates().groupby(['BIN','apt_num']).filter(lambda df: len(df)>1)

,BIN,rooms,bedrooms,bath,apt_num,amenities,no_proximity_newdev,BoroID,Boro
126,3154052.0,3 rooms,studio,1 bath,3,[],True,3,BROOKLYN
130,3154052.0,2 rooms,studio,1 bath,3,[],True,3,BROOKLYN
1088,3080174.0,5 rooms,3 beds,1 bath,4,[],False,3,BROOKLYN
1108,3080174.0,4 rooms,3 beds,1 bath,7,[],False,3,BROOKLYN
1142,3080174.0,4 rooms,3 beds,1 bath,4,[],False,3,BROOKLYN
1162,3080174.0,5 rooms,3 beds,1 bath,7,[],False,3,BROOKLYN


In [57]:
apartments_cleaned = apart_join[['BIN', 'rooms', 'bedrooms','bath', 'apt_num', 'amenities', 'no_proximity_newdev', 'BoroID',
       'Boro']].drop_duplicates().drop([130,1108,1142])

In [91]:
aparts_price_change = perc_diff.merge(apartments_cleaned, on = ['BIN','apt_num'])

In [103]:

import re 
aparts_price_change = pre_post_prices

In [100]:
aparts_price_change['rooms'] = aparts_price_change.rooms.str.extract('(^\d*)')

In [101]:
aparts_price_change['bath'] = aparts_price_change['bath'].apply(lambda x: re.findall('\d*\.?\d+',x)[0])

In [102]:
aparts_price_change['bedrooms'] = aparts_price_change['bedrooms'].apply(lambda x: 0 if 'studio' in x else  re.findall('\d*\.?\d+',x)[0] )

TypeError: argument of type 'int' is not iterable

In [109]:
aparts_price_change

,rooms,bedrooms,bath,amenities,no_proximity_newdev,BoroID,Boro,rent_price_cleaned,amenities_fl,pre_flag
2,2,0,1,"['Live-in Super', 'Virtual Doorman', 'NYC Stor...",True,3,BROOKLYN,1900,1,1
9,3,1,1,"['Gym', 'Laundry in Building', 'Live-in Super'...",True,1,MANHATTAN,2210,1,1
13,4,2,2.5,"['Gym', 'NYC Storm Zone 1', 'Doorman', 'Elevat...",True,3,BROOKLYN,4000,1,1
18,7,3,1.5,[],True,4,QUEENS,2000,0,1
24,2,0,1,[],True,1,MANHATTAN,1950,0,1
...,...,...,...,...,...,...,...,...,...,...
1543,4,1,1,[],True,4,QUEENS,1725,0,0
1547,4,2,2,"['Bike Room', 'Cold Storage', 'Green Building'...",False,3,BROOKLYN,3295,1,0
1555,4,3,1,[],True,3,BROOKLYN,2566.67,0,0
1560,5,3,1,[],True,3,BROOKLYN,2575,0,0


In [106]:
aparts_price_change = aparts_price_change.drop(columns = ['BIN','apt_num'])

In [107]:
aparts_price_change['pre_flag'] = aparts_price_change['year'].apply(lambda x: 1 if x < 2018 else 0)

In [108]:
aparts_price_change= aparts_price_change.drop(columns = ['year'])

In [110]:
aparts_price_change.to_csv('apartments_modeling_data.csv')